# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
 # Import the zip city data. Use dtype="object" to ensure all data is read in accurately.
City_Data = pd.read_csv(
    "../WeatherPy/City_Data.csv", dtype="object", encoding="utf-8")

# Visualize
City_Data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Thompson,75,CA,1619313930,46,55.7435,-97.8558,-3.0,3.6
1,1,Khani,100,GE,1619314086,93,41.9563,42.9566,8.0,1.03
2,2,Batagay-Alyta,100,RU,1619313992,72,67.8006,130.4114,-3.2,2.1
3,3,Binga,9,ZW,1619314086,68,-17.6203,27.3414,19.31,0.57
4,4,Kavieng,15,PG,1619313991,70,-2.5744,150.7967,29.39,4.54


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Configure gmaps with API key
gmaps.configure(api_key=g_key)

 # Store 'Lat' and 'Lng' into  locations 
locations = City_Data[["Latitude", "Longitude"]].astype(float)

Humidity = City_Data["Humidity"].astype(float)

In [4]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
City_Data = City_Data.dropna()
City_Data['Max Temp'] = City_Data['Max Temp'].astype(float)
City_Data['Wind Speed'] = City_Data['Wind Speed'].astype(float)
City_Data['Cloudiness'] = City_Data['Cloudiness'].astype(float)

City_Data_Clean = City_Data.drop(City_Data[(City_Data['Max Temp'] > 80) 
#                                            or (City_Data['Max Temp'] < 70) or (City_Data['Wind Speed'] > 10) or (City_Data['Cloudiness'] =! 0)].index)
# City_Data_Clean = City_Data.drop(City_Data[City_Data['Max Temp'] > 80].index, inplace = False)
# City_Data_Clean2 = City_Data.drop(City_Data[City_Data['Max Temp'] < 70].index, inplace = False)

City_Data_Clean

SyntaxError: invalid syntax (<ipython-input-23-10aa2366d159>, line 11)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
